<a href="https://colab.research.google.com/github/ProtossDragoon/Deep-Learning-with-Python/blob/main/Decorator_Tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Decorator Tutorial

선행되어야 하는 지식

- python 의 기본적인 문법들 (class 만드는 방법까지)
- 객체, attribute 에 대한 이해
- getter, setter 에 대한 이해

## Function in Python

- 이 챕터는 [이 내용 (stackoverflow)](https://stackoverflow.com/questions/739654/how-to-make-a-chain-of-function-decorators) 을 참고해서 한글로 번역한 내용입 니다.
- 우선, "python 에서 함수는 객체처럼 처리한다!" 라는 중요한 속성을 이해해야 합니다.

In [ ]:
def shout(word="yes"):
    return word.capitalize()+"!"

print(shout())
# outputs : 'Yes!'

# python 에서 함수는 객체니까! 함수를 다른 객체들처럼 그냥 변수에 담을 수 있습니다.
scream = shout

# 괄호가 없으니 함수를 호출(실행) 하는 것이 아닙니다.
# 이 코드는 함수 "shout" 를 변수 "scream" 에 담고 있습니다.
# 즉, "shout" 함수를 호출(실행) 하기 위해서, 아래와 같이 "scream" 변수를 이용할 수 있습니다.

print(scream())
# outputs : 'Yes!'

# 그렇다면 'shout' 변수를 제거하면 어떻게 될까요?
# 단순히, 'shout' 객체만 제거한 것이므로 (shout 는 함수이고, 함수도 객체라고 했으니까요!), 'scream' 객체를 통해 여전히 접근할 수 있습니다.

del shout
try:
    print(shout())
except NameError as e:
    print(e)
    # outputs: "name 'shout' is not defined"

print(scream())
# outputs: 'Yes!'

Yes!
Yes!
name 'shout' is not defined
Yes!


- python 에서는 함수 안에 함수를 정의할 수 있습니다.

In [ ]:
def talk():
    # "talk" 라는 함수 안에다가 다른 함수를 정의할 수 있습니다.
    def whisper(word="yes"):
        return word.lower()+"..."

    print(whisper())

# talk() 를 실행할 때마다, 새롭게 whilsper() 을 정의합니다. 즉, "whisper" 함수는 호출할 때마다 whisper() 을 새롭게 정의한다고 생각하면 됩니다.
talk()
# outputs: 
# "yes..."

# But "whisper" 함수는 "talk" 함수 밖에서 호출할 수 없습니다.

try:
    print(whisper())
except NameError as e:
    print(e)
    #outputs : "name 'whisper' is not defined"*
    #Python's functions are objects

yes...
name 'whisper' is not defined


- 여기까지 이해했다면, 함수도 객체니까, 함수를 return 할 수 있다는 생각이 들게 됩니다.
- return 도 된다면, 아주 당연히, parameter 로 전달도 가능하겠지요.

In [ ]:
def getTalk(kind="shout"):

    # "talk" 라는 함수 안에다가 다른 함수를 정의할 수 있습니다.
    def shout(word="yes"):
        return word.capitalize()+"!"

    # "talk" 라는 함수 안에다가 다른 함수를 정의할 수 있습니다.
    def whisper(word="yes") :
        return word.lower()+"...";

    # getTalk 함수 안에 정의된 다양한 함수들 중 하나를 return 해 봅시다..
    if kind == "shout":
        # return 하는 것은 객체라고 했습니다.
        return shout  
    else:
        return whisper


# 이제 이 요상한 녀석을 써 봅시다.
# getTalk() 을 실행합니다. 기본 파라미터가 shout 이니까, shout 함수 객체를 return 합니다.
talk = getTalk()      

# 진짜 객체인가 봅시다.
print(talk)
#outputs : <function shout at 0xb7ea817c>

# 우리가 함수를 실행시킬 때처럼, () 를 뒤에 붙여 줍니다.
print(talk())
#outputs : Yes!

# 다른 객체로 저장하지 않고 이렇게 사용할 수 있겠지요.
print(getTalk("whisper")())
#outputs : yes...



# 파라미터로 함수 전달하기
def doSomethingBefore(func): 
    print("I do something before then I call the function you gave me")
    print(func())

doSomethingBefore(scream)
#outputs: 
#I do something before then I call the function you gave me
#Yes!

<function getTalk.<locals>.shout at 0x7ff4705d11e0>
Yes!
yes...
I do something before then I call the function you gave me
Yes!


## Introduction of Decorator

### Decorator

> 데코레이터는, 다른 함수가 파라미터로 들어오는 함수를 의미합니다.

- 위에서 우리가 열심히 정의했던 내용을 유념하세요!
- 이 정의는 데코레이터에 대한 굉장히 정확한 정의입니다.
- 우선, 데코레이터를 활용할 수 있는 @ 를 배우기 전에, 직접 함수 객체를 활용해서 주고받는 데코레이터 구조를 만들어 봅시다.


> On the fly

- On-the-fly programming is the technique of modifying a program without stopping it.
- On the fly 란, 소스 코드를 run time 에 수정하는 기법을 의미입니다. 텍스트 에디터로 소스 코드를 수정하는 것이 아니고, 소스 코드를 통해 소스 코드를 변경할 수 있도록 합니다. 예를 들어 함수 A의 소스 코드가 B 였다면, C 로 바꾸어 주는 것과 같이 작동하도록 만들어 줍니다. 단순 if-else 문보다 훨씬 유연하게 동작하도록 만들 수 있습니다.
- C언어에 익숙하신 분이라면, 매크로처럼 소스코드가 바뀌며 동작한다고 생각하시면 됩니다. 물론 매크로는, 전처리 time 에 적용되는 기법이지만, 이는 run time 에 적용되는 소스코드 수정 기법이라는 점에서 더욱 유연합니다.
- 정확한 의미는 코드를 통해 알아봅시다.

In [ ]:
# 데코레이터는, 다른 함수가 파라미터로 들어오는 함수를 의미합니다.
def my_shiny_new_decorator(a_function_to_decorate):

    # 데코레이터 내부에는 run time 에 실행방법이 제어되는 함수 : wrapper 이라고 하는 놈을 정의합니다.
    # 이 wrapper 함수는 우리가 파라미터로 받은 또는 우리가 실행하고자 하는  "a_function_to_decorate" 함수에 대한 코드를 감싸게 될 함수입니다.
    # so it can execute code before and after it.
    def the_wrapper_around_the_original_function():

        # Put here the code you want to be executed BEFORE the original function is called
        # 원래 우리가 실행하고자 하는 함수 a_function_to_decorate 전에 실행하고자 하는 코드를 작성합니다.
        print("Before the function runs")

        # Call the function here (using parentheses)
        # parameter 로 받은 함수를 불러내는 부분입니다!
        a_function_to_decorate()

        # Put here the code you want to be executed AFTER the original function is called
        # 원래 우리가 실행하고자 하는 함수 a_function_to_decorate 이후에 실행하고자 하는 코드를 작성합니다.
        print("After the function runs")

    # 이 시점에 아직 a_function_to_decorate 함수는 아직 실행되지 않은 상태입니다. 당연하죠. the_wrapper_around_the_original_function 가 호출되지 않았으니까요.
    # 우리는 방금 만든 wrapper 함수 the_wrapper_around_the_original_function 객체를 그냥 return 할 것입니다.
    # wrapper 함수는, 우리가 파라미터로 받은, 또는 우리가 실행하고자 하는, 우리가 잘 감싸고자 하는 a_function_to_decorate 코드를 품고 있지요.
    # 이제 decorator 을 실행할 준비를 마치게 됩니다!
    return the_wrapper_around_the_original_function



# 그냥 평소에 우리가 만들듯이 만드는 함수 a_stand_alone_function 를 정의했다고 생각해 봅시다.
# 우리는 이 굉장히 허접한 함수를 절대 수정하지 않을 것입니다. 
def a_stand_alone_function():
    print("I am a stand alone function, don't you dare modify me")

a_stand_alone_function() 
#outputs: I am a stand alone function, don't you dare modify me


# 자, 우리는 이제 a_stand_alone_function 의 함수의 기능을 추가하기 위해, a_stand_alone_function 의 소스코드를 수정하지 않을 수 있을까요?
# a_stand_alone_function 함수 객체를 decorator 의 인자로 넣어 줍시다!
# a_stand_alone_function 함수는 decorator 의 wrapper 로 인해 매우 유연하게 포장될 것입니다.

a_stand_alone_function_decorated = my_shiny_new_decorator(a_stand_alone_function)
a_stand_alone_function_decorated()
#outputs:
#Before the function runs
#I am a stand alone function, don't you dare modify me
#After the function runs

I am a stand alone function, don't you dare modify me
Before the function runs
I am a stand alone function, don't you dare modify me
After the function runs


Now, you probably want that every time you call a_stand_alone_function, 
- 우리가 이렇게 열심히 a_stand_alone_function 의 기능을 추가했습니다.
- 애써 열심히 만든 이 데코레이터 함수 ```a_stand_alone_function_decorated``` 가 있으니까 그냥 a_stand_alone_function 를 호출할 때마다, ```a_stand_alone_function_decorated``` 가 호출되게 할 수는 없는 걸까요?
- a_stand_alone_function 를 대입한 함수를 my_shiny_new_decorator 의 파라미터로 넘기고, 이를 다시 a_stand_alone_function 에 저장하면 됩니다. 

In [ ]:
a_stand_alone_function = my_shiny_new_decorator(a_stand_alone_function)
a_stand_alone_function()
#outputs:
#Before the function runs
#I am a stand alone function, don't you dare modify me
#After the function runs

### Decorator Syntax
자! 지금까지 일어났던 일들이 궁극적으로 Decorator 이 하는 일의 본질입니다!
- 본질을 알고 사용하면 외울 필요가 없습니다.
- python 에서는 @decorator 이 작업을 빠르게 할 수 있도록 도와 줍니다.
- @decorator_name 은 단지 my_shiny_new_decorator(a_stand_alone_function) 을 대신 작업해 준다고 생각하면 됩니다.




In [ ]:
@my_shiny_new_decorator
def another_stand_alone_function():
    print("Leave me alone")

another_stand_alone_function()  
#outputs:  
#Before the function runs
#Leave me alone
#After the function runs

Before the function runs
Leave me alone
After the function runs


### Multiple Decorator
- 데코레이터 여러 개를 겹쳐서 사용할 수 있습니다.
- 우선 본질의 코드를 보고, python shortcut 코드를 보도록 합시다.
- 바로 아래 코드는 여러개의 데코레이터를 사용한다는 것의 본질을 보여 줍니다.

In [ ]:
def bread(func):
    def wrapper():
        print("</''''''\>")
        func()
        print("<\______/>")
    return wrapper

def ingredients(func):
    def wrapper():
        print("#tomatoes#")
        func()
        print("~salad~")
    return wrapper

def sandwich(food="--ham--"):
    print(food)

sandwich()
#outputs: --ham--
sandwich = bread(ingredients(sandwich))
sandwich()
#outputs:
#</''''''\>
# #tomatoes#
# --ham--
# ~salad~
#<\______/>

--ham--
</''''''\>
#tomatoes#
--ham--
~salad~
<\______/>


- 본질을 보았으니 python shortcut 을 보도록 합시다.
- python shortcut 코드의 경우, 순서가 중요해집니다.
- original function 즉, def sandwich 에서 가까운 데코레이터가 먼저 실행됩니다.

In [ ]:
@bread
@ingredients
def sandwich(food="--ham--"):
    print(food)

sandwich()
#outputs:
#</''''''\>
# #tomatoes#
# --ham--
# ~salad~
#<\______/>

</''''''\>
#tomatoes#
--ham--
~salad~
<\______/>


In [ ]:
@ingredients
@bread
def strange_sandwich(food="--ham--"):
    print(food)

strange_sandwich()
#outputs:
##tomatoes#
#</''''''\>
# --ham--
#<\______/>
# ~salad~

#tomatoes#
</''''''\>
--ham--
<\______/>
~salad~


데코레이터는 특정 함수의 기능을 원래 함수에 대입하는 기능으로 사용하기 좋겠구나! 라는 느낌을 가져가야 합니다. 왜냐하면, hello_world() 라는 정해진 함수의 빈칸에, go() 라는 함수를 런타임에 넣어주는 것이기 때문입니다. 이를 잘 활용하면 빈칸을 만들고 그곳에 기능을 삽입하는 역할을 매우 쉽게 구현할 수 있습니다.

### Passing Arguments to decorating function
- decorator 함수에 인자를 전달해 보도록 합시다.
- 지금까지 decorator 함수에 함수의 객체만 파라미터로 넣어 주었습니다.

In [ ]:
# 뭐 대단한 게 아니고, 그냥 wrapper 함수에 파라미터를 넣을 수 있도록 해 주면 됩니다.

def a_decorator_passing_arguments(function_to_decorate):
    def a_wrapper_accepting_arguments(arg1, arg2):
        print("I got args! Look: {0}, {1}".format(arg1, arg2))
        function_to_decorate(arg1, arg2)
    return a_wrapper_accepting_arguments

# decorator 이 아니라 wrapper 에 인수를 달아 주면 되는 이유는,
# 우리가 decorator 의 return 을 통해 wrapper 함수 객체를 얻고, 우리는 wrapper 을 호출할 것이기 때문에,
# wrapper 에 argument (함수의 인자, 파라미터) 를 넣어주는 것이 타당하기 때문입니다.


# 데코레이터 장착

@a_decorator_passing_arguments
def print_full_name(first_name, last_name):
    print("My name is", first_name, last_name)

print_full_name("Peter", "Venkman")
# outputs:
#I got args! Look: Peter Venkman
#My name is Peter Venkman

I got args! Look: Peter, Venkman
My name is Peter Venkman


### Decorator Method (Class function)

- python 데코레이터는 class 의 멤버 함수를 위한 데코레이터에도 일관된 문법을 적용합니다.

In [ ]:
def method_friendly_decorator(method_to_decorate):
    def wrapper(self, lie):
        lie = lie - 3
        return method_to_decorate(self, lie)
    return wrapper


class Lucy(object):

    def __init__(self):
        self.age = 32

    @method_friendly_decorator
    def sayYourAge(self, lie):
        print("I am", self.age + lie, "what did you think?")

l = Lucy()
l.sayYourAge(-3)
#outputs: I am 26, what did you think?

I am 26 what did you think?


If you’re making general-purpose decorator--one you’ll apply to any function or method, no matter its arguments--then just use *args, **kwargs:

- 굉장히 일반적인 용도로 사용될 수 있느 (general-purpose) 데코레이터를 만들고 싶다면, 파라미터로 ```*args **kwargs``` 를 사용하면 됩니다. 
- 아래 예시입니다.

In [ ]:
def a_decorator_passing_arbitrary_arguments(function_to_decorate):

    # wrapper 함수 a_wrapper_accepting_arbitrary_arguments 는 *args 와 **kwargs 를 사용해서 굉장히 일반적인 파라미터들을 받을 수 있게 됩니다.
    def a_wrapper_accepting_arbitrary_arguments(*args, **kwargs):
        print("Do I have args?:")
        print(args)
        print(kwargs)
        # args 와 kwargs 는 각각 tuple 과 dictionary 자료형으로 데이터들을 담고 있습니다.
        # 이 자료들을 잘 끄집어내어 주어야 합니다.

        # 자료들을 자료형으로부터 끄집어내는 방법이 익숙하지 않으면 아래를 참고하세요. 
        # http://www.saltycrane.com/blog/2008/01/how-to-use-args-and-kwargs-in-python/
        function_to_decorate(*args, **kwargs)
    return a_wrapper_accepting_arbitrary_arguments

@a_decorator_passing_arbitrary_arguments
def function_with_no_argument(): # 인자로 아무것도 넣지 않는 경우
    print("Python is cool, no argument here.")

function_with_no_argument()
#outputs
#Do I have args?:
#()
#{}
#Python is cool, no argument here.


@a_decorator_passing_arbitrary_arguments
def function_with_arguments(a, b, c): # 인자로 여러 개의 인자를 넘기는 경우
    print(a, b, c)

function_with_arguments(1,2,3)
#outputs
#Do I have args?:
#(1, 2, 3)
#{}
#1 2 3 

@a_decorator_passing_arbitrary_arguments
def function_with_named_arguments(a, b, c, platypus="Why not ?"): # 인자로 keyword (여기서는 platypus 같은 것들) 을 붙여서 넘기는 경우
    print("Do {0}, {1} and {2} like platypus? {3}".format(a, b, c, platypus))

function_with_named_arguments("Bill", "Linus", "Steve", platypus="Indeed!")
#outputs
#Do I have args ? :
#('Bill', 'Linus', 'Steve')
#{'platypus': 'Indeed!'}
#Do Bill, Linus and Steve like platypus? Indeed!


class Mary(object):

    def __init__(self):
        self.age = 31

    @a_decorator_passing_arbitrary_arguments
    def sayYourAge(self, lie=-3):
        print("I am {0}, what did you think?".format(self.age + lie))

m = Mary()
m.sayYourAge()
#outputs
# Do I have args?:
#(<__main__.Mary object at 0xb7d303ac>,)
#{}
#I am 28, what did you think?

Do I have args?:
()
{}
Python is cool, no argument here.
Do I have args?:
(1, 2, 3)
{}
1 2 3
Do I have args?:
('Bill', 'Linus', 'Steve')
{'platypus': 'Indeed!'}
Do Bill, Linus and Steve like platypus? Indeed!
Do I have args?:
(<__main__.Mary object at 0x7fd5a41b6be0>,)
{}
I am 28, what did you think?


### Passing arguments to the decorator

- 지금까지는 decorator 함수에 인자를 추가하는 것에 대해서 배워 보았습니다.
- 그렇다면 이번에는 decorator 그 자체에 인자를 추가해 볼 수 없을까요?
- 우선 decorator 그 자체에 인자를 추가한다는 것이 이해가 잘 되지 않을 수 있습니다. 예를 들어 이런 의문들이요.
  - 음? 이미 decorator function 에 함수 객체를 인자로 사용하고 있었잖아요!
  - decorator 과 decorator function 이 다른 거였어요?!
- 우선 첫 번째 의문에 답을 하자면, 그 의심이 맞습니다. 이미 decorator function 은 함수 객체를 인자로 사용하고 있기 때문에, decorator 에 다른 인자를 넣어 줄 수가 없어요.
- 이에 대한 해결책을, 실제로 코드를 보면서 이해하도록 합시다.
- 새로운 개념을 받아들이기 전에, 기존 내용을 한 번만 더 복습하고 가도록 해요.

This can get somewhat twisted, since a decorator must accept a function as an argument. Therefore, you cannot pass the decorated function’s arguments directly to the decorator.

Before rushing to the solution, let’s write a little reminder:

In [ ]:
# decorator 은, 그냥 평범한 함수에요!
def my_decorator(func):
    print("I am an ordinary function")
    def wrapper():
        print("I am function returned by the decorator")
        func()
    return wrapper

# 그러므로,  "@" 같은 것 없어도 똑같이 구현할 수 있어요. @ 는 python 에서 decorator 을 쉽게 연결하기 위해 만들어 준 것 뿐이에요.

def lazy_function():
    print("zzzzzzzz")

lazy_function = my_decorator(lazy_function)
#outputs: I am an ordinary function

# 위 예제의 출력은 "I am an ordinary function", 입니다. 
# 왜냐하면, 변수 lazy_function 에다가 함수를 등록해주었을(함수 객체를 저장했을) 뿐, wrapper 함수 객체를 ()을 통해 실행하지 않았기 때문입니다.
# 함수를 등록한다는 것은 wrapper 함수의 객체를 return 한 결과를 변수에 저장하겠다는 것인데, 
# 이것을 하기 위해서는 my_decorator(func) 가 실행되어야 하고, 이 과정에서 나온 print("I am an ordinary function") 만 실행된 것입니다.
# 아래 예제도 마찬가지입니다. 그냥 함수를 등록할 뿐, 실행하고자 하는 함수 lazy_function 을 실행하지 않을 겁니다.

@my_decorator
def lazy_function():
    print("zzzzzzzz")

#outputs: I am an ordinary function

I am an ordinary function
I am an ordinary function


- 두 결과가 완벽히 똑같다는 것을 확인했지요?
- @ 를 쓴다는 것의 의미를 정확히 파악하는 것이 중요합니다.
- python 에게 "my_decorator" 라는 변수에 의해 labelled 된 함수를 호출하라고 명령하는 것과 같습니다.
- you are telling Python to call the function 'labelled by the variable "my_decorator"'.



In [ ]:
def decorator_maker():

    print("I make decorators! I am executed only once: "
          "when you make me create a decorator.")

    def my_decorator(func):

        print("I am a decorator! I am executed only when you decorate a function.")

        def wrapped():
            print("I am the wrapper around the decorated function. "
                  "I am called when you call the decorated function. "
                  "As the wrapper, I return the RESULT of the decorated function.")
            return func()

        print("As the decorator, I return the wrapped function.")

        return wrapped

    print("As a decorator maker, I return a decorator")
    return my_decorator


# decorator 을 만들어 봅시다.
new_decorator = decorator_maker()       
#outputs:
#I make decorators! I am executed only once: when you make me create a decorator.
#As a decorator maker, I return a decorator

# 그리고 이제, decorator 을 이용해서 함수에 다양한 기능을 추가하고 꾸며 봅시다.

def decorated_function():
    print("I am the decorated function.")

decorated_function = new_decorator(decorated_function)
#outputs:
#I am a decorator! I am executed only when you decorate a function.
#As the decorator, I return the wrapped function

# 이제 기능이 추가된 (decorate 된 함수를) 실행해 봅시다.
decorated_function()
#outputs:
#I am the wrapper around the decorated function. I am called when you call the decorated function.
#As the wrapper, I return the RESULT of the decorated function.
#I am the decorated function.

I make decorators! I am executed only once: when you make me create a decorator.
As a decorator maker, I return a decorator
I am a decorator! I am executed only when you decorate a function.
As the decorator, I return the wrapped function.
I am the wrapper around the decorated function. I am called when you call the decorated function. As the wrapper, I return the RESULT of the decorated function.
I am the decorated function.


In [ ]:
def decorated_function():
    print("I am the decorated function.")

decorated_function = decorator_maker()(decorated_function)
#outputs:
#I make decorators! I am executed only once: when you make me create a decorator.
#As a decorator maker, I return a decorator
#I am a decorator! I am executed only when you decorate a function.
#As the decorator, I return the wrapped function.

# Finally:
decorated_function()    
#outputs:
#I am the wrapper around the decorated function. I am called when you call the decorated function.
#As the wrapper, I return the RESULT of the decorated function.
#I am the decorated function.

In [ ]:
# decorator_maker 뒤에 괄호 () 가 붙었다는 것을 잘 보세요!
@decorator_maker()
def decorated_function():
    print("I am the decorated function.")
#outputs:
#I make decorators! I am executed only once: when you make me create a decorator.
#As a decorator maker, I return a decorator
#I am a decorator! I am executed only when you decorate a function.
#As the decorator, I return the wrapped function.

# Eventually: 
decorated_function()   
#outputs:
#I am the wrapper around the decorated function. I am called when you call the decorated function.
#As the wrapper, I return the RESULT of the decorated function.
#I am the decorated function.

I make decorators! I am executed only once: when you make me create a decorator.
As a decorator maker, I return a decorator
I am a decorator! I am executed only when you decorate a function.
As the decorator, I return the wrapped function.
I am the wrapper around the decorated function. I am called when you call the decorated function. As the wrapper, I return the RESULT of the decorated function.
I am the decorated function.


굉장히 신기한 광경이 벌어진 것을 알 수 있습니다. 바로 "@" 키워드를 활용해서 함수를 "실행" 했다는 겁니다. @의 의미를 조금 아시겠나요? 이해가 안 됐다면 코드를 돌이켜 보면서, 이것을 확실히 이해하고 가야 합니다.

- 자 이제 다시 돌아옵시다.
- 만약 우리가 on-the-fly 로 decorator 만들어낼 수 있다면 (당연히, @ 키워드를 이용해서, 함수를 실행하고, 이 return 값으로 decorator 객체를 받도록 할 것이라는 예상을 할 수 있어야 합니다.) 우리는 인자들을 데코레이터 함수에 넣을 수 있을 겁니다.
- 핵심 개념은 아래 코드와 같습니다.


```python
def function_name1(func):
  return something

@function_name1 # @function_name1 은,  function_name2 = function_name1(function_name2) 와 동치임.
def function_name2(arg):
  return something
```


```python
def function_name1(arg):
  def function_name3(func):
    return something
  return function_name3

@function_name1(arg) # @function_name1(arg) 은, function_name1(arg) 의 return 인 function_name3 함수 객체에
                     # function_name2 를 넣은 function_name2 = function_name3(function_name2) 와 동치임.
def function_name2(arg):
  return something
```


- 실행해 보면서 조금 더 느낌을 파악하도록 합시다.



In [ ]:
def decorator_maker_with_arguments(decorator_arg1, decorator_arg2):

    print("I make decorators! And I accept arguments : ",decorator_arg1, decorator_arg2)

    def my_decorator(func):
        # 이 함수의 print() 문을 보면, decorator_arg1 와 decorator_arg2 가 존재하는데, 이것은 decorator_maker_with_arguments 에서 받은 인자들입니다.
        # 어떻게 my_decorator 에서 사용할 수 있는지를 정확히 이해하기 위해서는, python 의 closure 에 대한 이해가 필요합니다.
        # closure 덕분에, 밖의 decorator_maker_with_arguments 에서 받은 인자들을 my_decorator 에서 사용할 수 있는지 정확히 설명할 수 있습니다.
        # 이에 대해 익숙하지 않을 수 있지만, 뭐 괜찮습니다. 그래도 정 궁금하면 https://stackoverflow.com/questions/13857/can-you-explain-closures-as-they-relate-to-python 을 보세요.
        print("I am the decorator. Somehow you passed me arguments :", decorator_arg1, decorator_arg2)

        # decorator_arg 인자들과 fucntion_arg 인자들이 헷갈리지 앟게 주의하세요~
        def wrapped(function_arg1, function_arg2) :
            print("I am the wrapper around the decorated function.\n"
                  "I can access all the variables\n"
                  "\t- from the decorator: {0} {1}\n"
                  "\t- from the function call: {2} {3}\n"
                  "Then I can pass them to the decorated function"
                  .format(decorator_arg1, decorator_arg2,
                          function_arg1, function_arg2))
            return func(function_arg1, function_arg2)

        return wrapped

    return my_decorator

@decorator_maker_with_arguments("Leonard", "Sheldon")
def decorated_function_with_arguments(function_arg1, function_arg2):
    print("I am the decorated function and only knows about my arguments::", function_arg1, function_arg2)

decorated_function_with_arguments("Rajesh", "Howard")
#outputs:
#I make decorators! And I accept arguments: Leonard Sheldon
#I am the decorator. Somehow you passed me arguments: Leonard Sheldon
#I am the wrapper around the decorated function. 
#I can access all the variables 
#   - from the decorator: Leonard Sheldon 
#   - from the function call: Rajesh Howard 
#Then I can pass them to the decorated function
#I am the decorated function and only knows about my arguments: Rajesh Howard

I make decorators! And I accept arguments :  Leonard Sheldon
I am the decorator. Somehow you passed me arguments : Leonard Sheldon
I am the wrapper around the decorated function.
I can access all the variables
	- from the decorator: Leonard Sheldon
	- from the function call: Rajesh Howard
Then I can pass them to the decorated function
I am the decorated function and only knows about my arguments:: Rajesh Howard


In [ ]:
c1 = "Penny"
c2 = "Leslie"

@decorator_maker_with_arguments("Leonard", c1)
def decorated_function_with_arguments(function_arg1, function_arg2):
    print("I am the decorated function and only knows about my arguments:", function_arg1, function_arg2)

decorated_function_with_arguments(c2, "Howard")
#outputs:
#I make decorators! And I accept arguments: Leonard Penny
#I am the decorator. Somehow you passed me arguments: Leonard Penny
#I am the wrapper around the decorated function. 
#I can access all the variables 
#   - from the decorator: Leonard Penny 
#   - from the function call: Leslie Howard 
#Then I can pass them to the decorated function
#I am the decorated function and only know about my arguments: Leslie Howard

I make decorators! And I accept arguments :  Leonard Penny
I am the decorator. Somehow you passed me arguments : Leonard Penny
I am the wrapper around the decorated function.
I can access all the variables
	- from the decorator: Leonard Penny
	- from the function call: Leslie Howard
Then I can pass them to the decorated function
I am the decorated function and only knows about my arguments: Leslie Howard


As you can see, you can pass arguments to the decorator like any function using this trick. You can even use *args, **kwargs if you wish. But remember decorators are called only once. Just when Python imports the script. You can't dynamically set the arguments afterwards. When you do "import x", the function is already decorated, so you can't change anything.

## functools.wraps()

functools.wraps() 는 무엇을 하는 함수일까요?

partial(update_wrapper, wrapped=wrapped, assigned=assigned, updated=updated)

그런데, 이 functools.wraps() 함수를 이해하려면 functools.partial() 이라는 함수와, functools.update_wrapper() 라는 함수를 잘 알아봐야 이해할 수 있을것 같습니다.



In [ ]:
# 작성 중

## property()

이 내용은 https://www.freecodecamp.org/news/python-property-decorator/ 을 번역한 내용입니다.

### Why we use property()

Let's say that this class is part of your program. You are modeling a house with a House class (at the moment, the class only has a price instance attribute defined):


In [ ]:
class House:

	def __init__(self, price):
		self.price = price

This instance attribute is public because its name doesn't have a leading underscore. Since the attribute is currently public, it is very likely that you and other developers in your team accessed and modified the attribute directly in other parts of the program using dot notation, like this:

```
obj = House(1000000)

# Access value
print(obj.price)

# Modify value
obj.price = -400.12
```

So far everything is working great, right? But let's say that you are asked to make this attribute protected (non-public) and validate the new value before assigning it. Specifically, you need to check if the value is a positive float. How would you do that? Let's see.

At this point, if you decide to add getters and setters, you and your team will probably panic ?. This is because each line of code that accesses or modifies the value of the attribute will have to be modified to call the getter or setter, respectively. Otherwise, the code will break ⚠️.

```
obj = House(1000000)

# Changed from obj.price
print(obj.get_price())

# Changed from obj.price = 40000
obj.set_price(-400.12)
```

But... Properties come to the rescue! With @property, you and your team will not need to modify any of those lines because you will able to add getters and setters "behind the scenes" without affecting the syntax that you used to access or modify the attribute when it was public.

Awesome, right?  




### property() Syntax

Specifically, you can define three methods for a property:

- A getter - to access the value of the attribute.
- A setter - to set the value of the attribute.
- A deleter - to delete the instance attribute.


Price is now "Protected"
Please note that the price attribute is now considered "protected" because we added a leading underscore to its name in self._price:

그런데, 이 property() 또한 decorator 로 사용이 가능하기 때문에, 결론은 아래와 같습니다.


In [ ]:
class House:

	def __init__(self, price):
		self._price = price

	@property
	def price(self):
		return self._price
	
	@price.setter
	def price(self, new_price):
		if new_price > 0 and isinstance(new_price, float):
			self._price = new_price
		else:
			print("Please enter a valid price")

	@price.deleter
	def price(self):
		del self._price

## Yolov1 Model (other implementation)

### Load Package

In [ ]:
import tensorflow as tf
print(tf.__version__)

2.3.0


### Loss

In [ ]:
from functools import reduce

def compose(*funcs):
    """Compose arbitrarily many functions, evaluated left to right.
    Reference: https://mathieularose.com/function-composition-in-python/
    """
    # return lambda x: reduce(lambda v, f: f(v), funcs, x)
    if funcs:
        return reduce(lambda f, g: lambda *a, **kw: g(f(*a, **kw)), funcs) # 몇개의 layer 이 하나의 layer 로 합쳐지든 상관없도록 동작하도록 만드려고 한듯.
    else:
        raise ValueError('Composition of empty sequence not supported.')


# Loss 레이어를 만들어 봅시다. 
# tensorflow 에서는 모든 것들이 node 이자 layer 이라고 생각하면 편합니다.
class Loss(tf.keras.layers.Layer):
    def __init__(self, num_classes=20, cell_size=7, boxes_per_cell=2, *args, **kwargs):

        super(Loss, self).__init__(*args, **kwargs)

        # 모델을 만드는 것에 필요한 memeber variable
        self.num_classes      = num_classes
        self.cell_size        = cell_size
        self.boxes_per_cell   = boxes_per_cell

        # loss function 에서 필요한 member variable
        self.object_scale     = 1.0
        self.noobject_scale   = 0.5
        self.class_scale      = 2.0
        self.coord_scale      = 5.0

    # x,y loss
    # w,h loss
    def regression_loss(self, regression_target, regression, object_mask):
      '''
      :param inputs:
      regression_target: 
      object_mask: 해당 bbox 에 object 가 있는지 없는지를 판단해 주는 것
      :return
      reg_loss: regression loss 텐서
      '''
        coord_mask = tf.expand_dims(object_mask, 4) # 가장 뒷쪽에 span. object_mask shape : [batch, w, h, ch]
        boxes_delta = coord_mask * (regression_target - regression)
        coord_loss = tf.reduce_mean(tf.reduce_sum(tf.square(boxes_delta), axis=[1, 2, 3, 4]),
                                    name='coord_loss') * self.coord_scale
        reg_loss = tf.constant(0,dtype=tf.float32)
        return reg_loss

    def classification_loss(self, labels,  classification, response):
        class_delta = response * (labels - classification)
        cls_loss = tf.reduce_mean(tf.reduce_sum(tf.square(class_delta), axis=[1, 2, 3]), name='cls_loss') * self.class_scale

        return cls_loss

    def confidence_loss(self,predict_scales, iou_predict_truth, object_mask):
        # calculate no_I tensor [CELL_SIZE, CELL_SIZE, BOXES_PER_CELL]
        noobject_mask = tf.ones_like(object_mask, dtype=tf.float32) - object_mask

        # object_loss
        object_delta = object_mask * (iou_predict_truth - predict_scales)
        object_loss = tf.reduce_mean(tf.reduce_sum(tf.square(object_delta), axis=[1, 2, 3]),  name='object_loss') * self.object_scale

        # noobject_loss
        noobject_delta = noobject_mask * predict_scales
        noobject_loss = tf.reduce_mean(tf.reduce_sum(tf.square(noobject_delta), axis=[1, 2, 3]), name='noobject_loss') * self.noobject_scale

        return [object_loss, noobject_loss]


    def call(self, inputs): # tf.keras.layers.layer 의 상속은 call method 를 구현한다.
        '''
        :param inputs:
        predicts: shape(None, 1470)
        labels shape(None, 7, 7, 25)
        :return:
        '''

        # self.loss([x, gt_boxes, image_shape])
        predicts, labels, image_shape = inputs

        # 첫번째 요소 : 각 cell 에 해당되는, 해당 cell 의 class 가 무엇일지에 대한 score
        index_classification = tf.multiply(tf.pow(self.cell_size, 2), self.num_classes)                     # [0 : (7 x 7 x class개수)]
        
        # 두번째 요소 : 각 cell 에 해당되는, 물체가 있을지에 대한 score
        index_confidence = tf.multiply(tf.pow(self.cell_size, 2), self.num_classes + self.boxes_per_cell)   # [(7 x 7 x class개수) : (7 x 7 x (num_classes + cell마다추론하는박스개수))]
        
        # 세번째 줄에 생략된 부분 : 각 cell 마다 할당되는 2개의 box 를 추정하는 부분
        # index_boxes [(7 x 7 x (num_classes + cell마다추론하는박스개수)) : 1470]


        predict_classes = tf.reshape(predicts[:, :index_classification], [-1, self.cell_size, self.cell_size, self.num_classes])                    # (Batch, 7x7xC)
        predict_scales  = tf.reshape(predicts[:, index_classification:index_confidence], [-1, self.cell_size, self.cell_size, self.boxes_per_cell]) # (Batch, 7x7xB)
        predict_boxes   = tf.reshape(predicts[:, index_confidence:], [-1, self.cell_size, self.cell_size, self.boxes_per_cell, 4])                  # (Batch, 7x7x2x4)
                                                
                                              # (batch, w, h, c)   # (-1, w, h, c)
        response              = tf.reshape(labels[:, :, :, 0],      [-1, self.cell_size, self.cell_size, 1])                          # object 가 있는지 없는지 검사하는 용도의 label. (Batch, 7x7x1)
        regression_labels     = tf.reshape(labels[:, :, :, 1:5],    [-1, self.cell_size, self.cell_size, 1, 4])                       # grid 의 center 로부터 xywh 을 확인하는 용도의 label. (Batch, 7x7x4)
        regression_labels     = tf.div(tf.tile(regression_labels,   [1, 1, 1, self.boxes_per_cell, 1]), tf.to_float(image_shape[0]))  # https://www.tensorflow.org/api_docs/python/tf/tile 이해하고 온다 실시. 쉽게 말하면 채널을 박스 개수만큼 복사한것. 모든 박스의 Groundtruth 는 똑같아야 하기 때문에. 
        classification_labels = labels[:, :, :, 5:]                                                                                   # 두말할 것도 없이 (Batch, 7x7x클래스종류). 여기서 단점. 박스는 cell 당 N 개 예측가능한데, 클래스는 하나여야함 ㅋㅋㅋㅋ

        offset = np.transpose(np.reshape(np.array(
                                        [np.arange(self.cell_size)] * self.cell_size * self.boxes_per_cell), # [[0, 1, 2, ... , 6] * 7 * 2 ] (1, 7x7x2)
                                        (self.boxes_per_cell, self.cell_size, self.cell_size)),              # (1, 7x7x2) ---> (2, 7, 7)
                              (1, 2, 0))
        offset = tf.constant(offset, dtype=tf.float32)
        offset = tf.reshape(offset, [1, self.cell_size, self.cell_size, self.boxes_per_cell])

        regression = tf.stack([(predict_boxes[:, :, :, :, 0] + offset) / self.cell_size,
                               (predict_boxes[:, :, :, :, 1] + tf.transpose(offset, (0, 2, 1, 3))) / self.cell_size,
                               tf.square(predict_boxes[:, :, :, :, 2]),
                               tf.square(predict_boxes[:, :, :, :, 3])])
        regression = tf.transpose(regression, [1, 2, 3, 4, 0])

        iou_predict_truth = self.calc_iou(regression, regression_labels)

        # calculate I tensor [BATCH_SIZE, CELL_SIZE, CELL_SIZE, BOXES_PER_CELL]
        object_mask = tf.reduce_max(iou_predict_truth, 3, keep_dims=True)
        object_mask = tf.cast((iou_predict_truth >= object_mask), tf.float32) * response

        regression_target = tf.stack([regression_labels[:, :, :, :, 0] * self.cell_size - offset,
                               regression_labels[:, :, :, :, 1] * self.cell_size - tf.transpose(offset, (0, 2, 1, 3)),
                               tf.sqrt(regression_labels[:, :, :, :, 2]),
                               tf.sqrt(regression_labels[:, :, :, :, 3])])
        regression_target = tf.transpose(regression_target, [1, 2, 3, 4, 0])

        # regression loss (localization loss) coord_loss
        coord_loss = self.regression_loss(regression_target, predict_boxes, object_mask)

        # confidence loss
        object_loss, noobject_loss = self.confidence_loss(predict_scales, iou_predict_truth, object_mask)

        # classification loss
        cls_loss = self.classification_loss(classification_labels, predict_classes, response)

        self.add_loss(cls_loss)
        self.add_loss(object_loss)
        self.add_loss(noobject_loss)
        self.add_loss(coord_loss)

        return [coord_loss, object_loss, noobject_loss, cls_loss]

    def calc_iou(self, boxes1, boxes2, scope='iou'):
        """calculate ious
        Args:
          boxes1: 4-D tensor [CELL_SIZE, CELL_SIZE, BOXES_PER_CELL, 4]  ====> (x_center, y_center, w, h)
          boxes2: 1-D tensor [CELL_SIZE, CELL_SIZE, BOXES_PER_CELL, 4] ===> (x_center, y_center, w, h)
        Return:
          iou: 3-D tensor [CELL_SIZE, CELL_SIZE, BOXES_PER_CELL]
        """
        with tf.variable_scope(scope):
            boxes1 = tf.stack([boxes1[:, :, :, :, 0] - boxes1[:, :, :, :, 2] / 2.0,
                               boxes1[:, :, :, :, 1] - boxes1[:, :, :, :, 3] / 2.0,
                               boxes1[:, :, :, :, 0] + boxes1[:, :, :, :, 2] / 2.0,
                               boxes1[:, :, :, :, 1] + boxes1[:, :, :, :, 3] / 2.0])
            boxes1 = tf.transpose(boxes1, [1, 2, 3, 4, 0])

            boxes2 = tf.stack([boxes2[:, :, :, :, 0] - boxes2[:, :, :, :, 2] / 2.0,
                               boxes2[:, :, :, :, 1] - boxes2[:, :, :, :, 3] / 2.0,
                               boxes2[:, :, :, :, 0] + boxes2[:, :, :, :, 2] / 2.0,
                               boxes2[:, :, :, :, 1] + boxes2[:, :, :, :, 3] / 2.0])
            boxes2 = tf.transpose(boxes2, [1, 2, 3, 4, 0])

            # calculate the left up point & right down point
            lu = tf.maximum(boxes1[:, :, :, :, :2], boxes2[:, :, :, :, :2])
            rd = tf.minimum(boxes1[:, :, :, :, 2:], boxes2[:, :, :, :, 2:])

            # intersection
            intersection = tf.maximum(0.0, rd - lu)
            inter_square = intersection[:, :, :, :, 0] * intersection[:, :, :, :, 1]

            # calculate the boxs1 square and boxs2 square
            square1 = (boxes1[:, :, :, :, 2] - boxes1[:, :, :, :, 0]) * \
                      (boxes1[:, :, :, :, 3] - boxes1[:, :, :, :, 1])
            square2 = (boxes2[:, :, :, :, 2] - boxes2[:, :, :, :, 0]) * \
                      (boxes2[:, :, :, :, 3] - boxes2[:, :, :, :, 1])

            union_square = tf.maximum(square1 + square2 - inter_square, 1e-10)

        return tf.clip_by_value(inter_square / union_square, 0.0, 1.0)

    def compute_output_shape(self, input_shape):
        return [(1,), (1,), (1,), (1,)]

    def compute_mask(self, inputs, mask=None):
        return [None, None, None, None]

    def get_config(self):
        return {
            'num_classes' : self.num_classes,
            'cell_size'   : self.cell_size,
        }  

class Dimensions(tf.keras.layers.Layer):
    def __call__(self, inputs, **kwargs):
        return tf.keras.backend.shape(inputs)[1:3]

    def compute_output_shape(self, input_shape):
        return (input_shape[0], 2,)

NameError: ignored

### Define Model

In [ ]:
from functools import wraps

class Yolo(object):
    def __init__(self, training=True, num_classes=21, weights=None):
        '''
        Fast RCNN introduced in Faster R-CNN.
        '''
        super(Yolo, self).__init__()

        self.training = training
        self.num_classes = num_classes

        self.network()

    # @wraps(tf.keras.layers.Conv2D) 은, 이 wrapper function 을 정의하는 데 tf.keras.layers.Conv2D 의 메타데이터 (종속성, 함수명 등) 를 가져오겠다는 의미입니다.
    # https://iissgnoheci.tistory.com/8 을 참고.
    # 근데 여기서 wrapper 이 꼭 필요한가?
    @wraps(tf.keras.layers.Conv2D)
    def _Conv2D(self,*args, **kwargs): # 이 wrapper 은 tf.keras.layers.conv2D 객체를 반환하는데, _kwargs 를 활용하여 **kwargs 로 전달, 
        """Wrapper to set Darknet parameters for Convolution2D."""
        _kwargs = {'kernel_regularizer': tf.keras.regularizers.l2(5e-4)}
        _kwargs['padding'] = 'valid' if kwargs.get('strides') == (2, 2) else 'same'
        _kwargs.update(kwargs)
        return tf.keras.layers.Conv2D(*args, **_kwargs) # 이렇게 리스트와 딕셔너리에 *과 **을 붙여서 패스하면, 같은 이름의 것들이 자동으로 매칭되어 버림. 개꿀팁
        # 이 wrapper 은, 원래 _Conv2D 의 역할처럼, Conv2D(_Conv2D) 는 init() 을 호출함, return 으로 Conv2D 객체를 return 할 것이다.
    

    def _Conv2D_BN_Leaky(self, *args, **kwargs):
        """Darknet Convolution2D followed by BatchNormalization and LeakyReLU."""
        no_bias_kwargs = {'use_bias': False}
        no_bias_kwargs.update(kwargs)
        return compose(
            self._Conv2D(*args, **no_bias_kwargs),
            tf.keras.layers.BatchNormalization(), # no bn in yolo v1
            # ReLU())
            tf.keras.layers.LeakyReLU(alpha=0.1))

    def network(self):
        # Convolution block 0
        self.out_0      = self._Conv2D_BN_Leaky(64, kernel_size=(7,7), strides=(1,1), name='conv_0')
        self.pooling_0  = tf.keras.layers.MaxPooling2D(pool_size=(2,2), strides=2, name='pooling_0')

        # Convolution block 1
        self.out_1      = self._Conv2D_BN_Leaky(192, kernel_size=(3, 3), strides=(1, 1), name='conv_1')
        self.pooling_1  = tf.keras.layers.MaxPooling2D(pool_size=(2,2), strides=2, name='pooling_1')

        # Convolution block 2
        self.out_2      = self._Conv2D_BN_Leaky(128, kernel_size=(1, 1), strides=(1, 1), name='conv_2')
        self.out_3      = self._Conv2D_BN_Leaky(256, kernel_size=(3, 3), strides=(1, 1), name='conv_3')
        self.out_4      = self._Conv2D_BN_Leaky(256, kernel_size=(1, 1), strides=(1, 1), name='conv_4')
        self.out_5      = self._Conv2D_BN_Leaky(512, kernel_size=(3, 3), strides=(1, 1), name='conv_5')
        self.pooling_2  = tf.keras.layers.MaxPooling2D(pool_size=(2,2), strides=2, name='pooling_2')

        # Convolution block 3
        self.out_6      = self._Conv2D_BN_Leaky(256, kernel_size=(1, 1), strides=(1, 1), name='conv_6')
        self.out_7      = self._Conv2D_BN_Leaky(512, kernel_size=(3, 3), strides=(1, 1), name='conv_7')
        self.out_8      = self._Conv2D_BN_Leaky(256, kernel_size=(1, 1), strides=(1, 1), name='conv_8')
        self.out_9      = self._Conv2D_BN_Leaky(512, kernel_size=(3, 3), strides=(1, 1), name='conv_9')
        self.out_10     = self._Conv2D_BN_Leaky(256, kernel_size=(1, 1), strides=(1, 1), name='conv_10')
        self.out_11     = self._Conv2D_BN_Leaky(512, kernel_size=(3, 3), strides=(1, 1), name='conv_11')
        self.out_12     = self._Conv2D_BN_Leaky(256, kernel_size=(1, 1), strides=(1, 1), name='conv_12')
        self.out_13     = self._Conv2D_BN_Leaky(512, kernel_size=(3, 3), strides=(1, 1), name='conv_13')
        self.out_14     = self._Conv2D_BN_Leaky(512, kernel_size=(1, 1), strides=(1, 1), name='conv_14')
        self.out_15     = self._Conv2D_BN_Leaky(1024,kernel_size=(3, 3), strides=(1, 1), name='conv_15')
        self.pooling_3  = tf.keras.layers.MaxPooling2D(pool_size=(2,2), strides=2, name='pooling_3')

        # Convolution block 4
        self.out_16     = self._Conv2D_BN_Leaky(512, kernel_size=(1, 1), strides=(1, 1), name='conv_16')
        self.out_17     = self._Conv2D_BN_Leaky(1024, kernel_size=(3, 3), strides=(1, 1), name='conv_17')
        self.out_18     = self._Conv2D_BN_Leaky(512, kernel_size=(1, 1), strides=(1, 1), name='conv_18')
        self.out_19     = self._Conv2D_BN_Leaky(1024, kernel_size=(3, 3), strides=(1, 1), name='conv_19')
        self.out_20     = self._Conv2D_BN_Leaky(1024, kernel_size=(3, 3), strides=(1, 1), name='conv_20')
        self.out_21     = self._Conv2D_BN_Leaky(1024, kernel_size=(3, 3), strides=(2, 2), name='conv_21')

        # Convolution block 5
        self.out_22     = self._Conv2D_BN_Leaky(1024, kernel_size=(3, 3), strides=(1, 1), name='conv_22')
        self.out_23     = self._Conv2D_BN_Leaky(1024, kernel_size=(3, 3), strides=(1, 1), name='conv_23')

        self.flatten = tf.keras.layers.Flatten(name='Flatten')
        self.fc_0 = tf.keras.layers.Dense(units=512,name='fc_0')
        self.fc_1 = tf.keras.layers.Dense(units=4096,name='fc_1')
        self.fc_2 = tf.keras.layers.Dense(units=7*7*30, name='fc_2')

        if self.training:
            self.loss = Loss(num_classes=self.num_classes)

    def __call__(self, inputs, mask=None):
        if self.training:
            image, gt_boxes = inputs
            image_shape = Dimensions()(image)
        else:
            image = inputs

        classification = None
        regression = None
        loss = None

        x = self.out_0(image)
        x = self.pooling_0(x)

        x = self.out_1(x)
        x = self.pooling_1(x)

        x = self.out_2(x)
        x = self.out_3(x)
        x = self.out_4(x)
        x = self.out_5(x)
        x = self.pooling_2(x)

        x = self.out_6(x)
        x = self.out_7(x)
        x = self.out_8(x)
        x = self.out_9(x)
        x = self.out_10(x)
        x = self.out_11(x)
        x = self.out_12(x)
        x = self.out_13(x)
        x = self.out_14(x)
        x = self.out_15(x)
        x = self.pooling_3(x)

        x = self.out_16(x)
        x = self.out_17(x)
        x = self.out_18(x)
        x = self.out_19(x)
        x = self.out_20(x)
        x = self.out_21(x)
        x = self.out_22(x)
        x = self.out_23(x)
        x = self.flatten(x)
        x = self.fc_0(x)
        x = self.fc_1(x)
        x = self.fc_2(x)

        if self.training:
            coord_loss, object_loss, noobject_loss, cls_loss = self.loss([x, gt_boxes, image_shape])

        return coord_loss, object_loss, noobject_loss, cls_loss, x

training session 에는
- self.loss (여기에는 Loss 직접 만든 클래스 객체가 저장되어 있음.) 함수를 거친 output
- coord_loss, object_loss, noobject_loss, cls_loss, 
- x

test session 에는
- coord_loss, object_loss, noobject_loss, cls_loss
- x

그렇다면 Loss 클래스에는 무슨 일이 일어나고 있을까.


### PascalVOC Data Generator

In [ ]:
import numpy as np
import random
import threading
import warnings
from PIL import Image

def read_image_bgr(path):
    '''
    :param path:
    :return: (h, w, 3)
    '''
    try:
        image = np.asarray(Image.open(path).convert('RGB'))
    except Exception as ex:
        print(path)

    return image[:, :, ::-1].copy()

def preprocess_image(x):
    # mostly identical to "https://github.com/fchollet/keras/blob/master/keras/applications/imagenet_utils.py"
    # except for converting RGB -> BGR since we assume BGR already
    x = x.astype(tf.keras.backend.floatx())
    if keras.backend.image_data_format() == 'channels_first':
        if x.ndim == 3:
            x[0, :, :] -= 103.939
            x[1, :, :] -= 116.779
            x[2, :, :] -= 123.68
        else:
            x[:, 0, :, :] -= 103.939
            x[:, 1, :, :] -= 116.779
            x[:, 2, :, :] -= 123.68
    else:
        x[..., 0] -= 103.939
        x[..., 1] -= 116.779
        x[..., 2] -= 123.68

    return x

def resize_image(image, min_side=448, max_side=448):
    '''
    resize image to dsize
    :param img: input (h, w, 3) = (rows, cols, 3)
    :param size:
    :return: out (h, w, 3)
    '''
    (h, w, _) = image.shape

    scale = np.asarray((min_side, max_side),dtype=float) / np.asarray((h, w),dtype=float)

    # resize the image with the computed scale
    # cv2.resize(image, (w, h))
    img = cv2.resize(image, (min_side, max_side))

    return img, scale

class Generator(object):
    def __init__(
        self,
        transform_generator = None,
        batch_size=2,
        group_method='random',  # one of 'none', 'random', 'ratio'
        shuffle_groups=True,
        image_min_side=448,
        image_max_side=448,
        cell_size=7,
        transform_parameters=None,
    ):
        self.transform_generator    = transform_generator
        self.batch_size             = int(batch_size)
        self.group_method           = group_method
        self.shuffle_groups         = shuffle_groups
        self.image_min_side         = image_min_side
        self.image_max_side         = image_max_side
        self.cell_size              = cell_size
        self.transform_parameters   = transform_parameters

        self.group_index = 0
        self.lock = threading.Lock()
        self.group_images()


    def size(self):
        raise NotImplementedError('size method not implemented')

    def num_classes(self):
        raise NotImplementedError('num_classes method not implemented')

    def name_to_label(self, name):
        raise NotImplementedError('name_to_label method not implemented')

    def label_to_name(self, label):
        raise NotImplementedError('label_to_name method not implemented')

    def image_aspect_ratio(self, image_index):
        raise NotImplementedError('image_aspect_ratio method not implemented')

    def load_image(self, image_index):
        raise NotImplementedError('load_image method not implemented')

    def load_annotations(self, image_index):
        raise NotImplementedError('load_annotations method not implemented')

    def load_annotations_group(self, group):
        return [self.load_annotations(image_index) for image_index in group]

    def filter_annotations(self, image_group, annotations_group, group):
        # test all annotations
        for index, (image, annotations) in enumerate(zip(image_group, annotations_group)):
            assert(isinstance(annotations, np.ndarray)), '\'load_annotations\' should return a list of numpy arrays, received: {}'.format(type(annotations))

            # test x2 < x1 | y2 < y1 | x1 < 0 | y1 < 0 | x2 <= 0 | y2 <= 0 | x2 >= image.shape[1] | y2 >= image.shape[0]
            invalid_indices = np.where(
                (annotations[:, 2] <= annotations[:, 0]) |
                (annotations[:, 3] <= annotations[:, 1]) |
                (annotations[:, 0] < 0) |
                (annotations[:, 1] < 0) |
                (annotations[:, 2] > image.shape[1]) |
                (annotations[:, 3] > image.shape[0])
            )[0]

            # delete invalid indices
            if len(invalid_indices):
                warnings.warn('Image with id {} (shape {}) contains the following invalid boxes: {}.'.format(
                    group[index],
                    image.shape,
                    [annotations[invalid_index, :] for invalid_index in invalid_indices]
                ))
                annotations_group[index] = np.delete(annotations, invalid_indices, axis=0)

        return image_group, annotations_group

    def load_image_group(self, group):
        return [self.load_image(image_index) for image_index in group] # 가장 밖의 axis 에다가 image 를 차곡차곡 하는 모습. [ [img], [img], [img], [img], ... ]

    def random_transform_group_entry(self, image, annotations):
        # randomly transform both image and annotations
        if self.transform_generator:
            pass

        return image, annotations

    def resize_image(self, image):
        return resize_image(image, min_side=self.image_min_side, max_side=self.image_max_side)

    def preprocess_image(self, image):
        return preprocess_image(image)

    def preprocess_group_entry(self, image, annotations):
        # preprocess the image
        image = self.preprocess_image(image)

        # randomly transform image and annotations
        # image, annotations = self.random_transform_group_entry(image, annotations) # not implemented

        # resize image
        image, image_scale = self.resize_image(image)

        # apply resizing to annotations too
        annotations[:, 0:4:2] *= image_scale[1]
        annotations[:, 1:4:2] *= image_scale[0]

        return image, annotations

    def preprocess_group(self, image_group, annotations_group):
        for index, (image, annotations) in enumerate(zip(image_group, annotations_group)):
            # preprocess a single group entry
            image, annotations = self.preprocess_group_entry(image, annotations)

            # copy processed data back to group
            image_group[index]       = image
            annotations_group[index] = annotations

        return image_group, annotations_group

    def group_images(self): # init() 에서 실행됨.
        # determine the order of the images
        order = list(range(self.size())) # 총 데이터의 개수를 range 로 바꿔서, 0~데이터개수-1 리스트로 정리. 이때 size() 는 오버라이딩됨.
        if self.group_method == 'random':
            random.shuffle(order)
        elif self.group_method == 'ratio':
            order.sort(key=lambda x: self.image_aspect_ratio(x))

        # divide into groups, one group = one batch
        # 1 : [order[x % 데이터개수] for x in range(i, i + self.batch_size)] 이때 i 는, 0, 15, 31, ... 이런 식으로 변함.
          # order 이 이미 한번 섞였든 말든 상관없이
        self.groups = [[order[x % len(order)] for x in range(i, i + self.batch_size)] for i in range(0, len(order), self.batch_size)]

    def compute_inputs(self, image_group):
        # get the max image shape
        max_shape = tuple(max(image.shape[x] for image in image_group) for x in range(3))

        # construct an image batch object
        image_batch = np.zeros((self.batch_size,) + max_shape, dtype=keras.backend.floatx())

        # copy all images to the upper left part of the image batch object
        for image_index, image in enumerate(image_group):
            image_batch[image_index, :image.shape[0], :image.shape[1], :image.shape[2]] = image

        return image_batch

    def compute_targets(self, image_group, annotations_group):
        # same size for all batch image
        h, w, _ = image_group[0].shape

        targets =  np.zeros((len(annotations_group), self.cell_size, self.cell_size, 25))
        for annotation_index, annotation in enumerate(annotations_group):
            label = np.zeros((self.cell_size, self.cell_size, 25))
            
            box_chw =np.stack([
                (annotation[:,0] + annotation[:,2])/2,
                (annotation[:,1] + annotation[:,3])/2,
                (annotation[:,2] - annotation[:,0]),
                (annotation[:,3] - annotation[:,1])], axis=1)
            # cls_ind = [self.label_to_name(label) for label in annotation[:, 4]]
            cls_ind = np.int32(annotation[:, 4])

            # x_ind， y_ind
            x_ind = np.int32(box_chw[:, 0] * self.cell_size / w)
            y_ind = np.int32(box_chw[:, 1] * self.cell_size / h)

            #Each grid cell predicts only one object
            index =  np.where(label[y_ind, x_ind,0]!=1)
            if len(index):
                label[y_ind[index], x_ind[index], 0] = 1
                label[y_ind[index], x_ind[index], 1:5] = box_chw[index,...]
                label[y_ind[index], x_ind[index], 5+cls_ind[index]] = 1

            targets[annotation_index] = label

        return np.asarray(targets)

    def compute_input_output(self, group):
        # load images and annotations
        image_group       = self.load_image_group(group)
        annotations_group = self.load_annotations_group(group)

        # check validity of annotations
        image_group, annotations_group = self.filter_annotations(image_group, annotations_group, group)

        # perform preprocessing steps
        image_group, annotations_group = self.preprocess_group(image_group, annotations_group)

        # compute network inputs
        inputs = self.compute_inputs(image_group)

        # compute network targets
        targets = self.compute_targets(image_group, annotations_group)

        return [inputs, targets], None

    def __next__(self): # __next()__ 를 구현하면 generator 을 만들 수 있다구. 이 함수는 custom 함수 next() 의 리턴값을 를 리턴.
        return self.next()
    def next(self):
        # advance the group index
        with self.lock: # self.lock.acquire()
            if self.group_index == 0 and self.shuffle_groups:
                # shuffle groups at start of epoch
                random.shuffle(self.groups)
            group = self.groups[self.group_index]
            self.group_index = (self.group_index + 1) % len(self.groups)
            # self.lock.release()

        return self.compute_input_output(group)

group (그룹은 1batch 랑 동일함. 데이터는, int 형식으로, 예를 들어 batch size 5 라면, ) - group index 이 있고 <br> 
image_group ([Batch, W, H, Ch] 이미지 데이터 Tensor) <br>
annotations_group () 이 있음. <br>

- \_\_init()__ 에서 group 이런 것들을 초기화해줌.
- \_\_next()__ 를 구현하면 generator 을 만들 수 있다구. 이 함수는 custom 함수 next() 의 리턴값을 를 리턴.
  - next()
    - group = self.groups[self.group_index]
    - compute_input_output(group)
      - image_group = load_image_group(group) 
        - 아래 코드를 그룹 모든 원소들에 대해서 실행
        - load_image(self, image_index) << **상속받은 클래스에서 구현**
      - annotations_group = load_annotations_group(group)
        - 아래 코드를 그룹 모드 원소들에 대해서 실행
        - load_annotations(image_index) << **상속받은 클래스에서 구현**
      - filter_annotations(image_group, annotations_group, group) 
      - preprocess_group(image_group, annotations_group)
      - inputs = compute_inputs(image_group)
      - targets = compute_targets(image_group, annotations_group)
    - **return [input, targets], None**


In [ ]:
from ..utils.image import read_image_bgr

import os
import numpy as np
from six import raise_from
from PIL import Image

try:
    import xml.etree.cElementTree as ET
except ImportError:
    import xml.etree.ElementTree as ET

voc_classes = {
    'aeroplane'   : 0,
    'bicycle'     : 1,
    'bird'        : 2,
    'boat'        : 3,
    'bottle'      : 4,
    'bus'         : 5,
    'car'         : 6,
    'cat'         : 7,
    'chair'       : 8,
    'cow'         : 9,
    'diningtable' : 10,
    'dog'         : 11,
    'horse'       : 12,
    'motorbike'   : 13,
    'person'      : 14,
    'pottedplant'   : 15,
    'sheep'       : 16,
    'sofa'        : 17,
    'train'       : 18,
    'tvmonitor'   : 19
}

def _findNode(parent, name, debug_name = None, parse = None):
    if debug_name is None:
        debug_name = name

    result = parent.find(name)
    if result is None:
        raise ValueError('missing element \'{}\''.format(debug_name))
    if parse is not None:
        try:
            return parse(result.text)
        except ValueError as e:
            raise_from(ValueError('illegal value for \'{}\': {}'.format(debug_name, e)), None)
    return result


class PascalVocGenerator(Generator):
    def __init__(
        self,
        data_dir,
        set_name,
        classes=voc_classes,
        image_extension='.jpg',
        skip_truncated=False,
        skip_difficult=False,
        **kwargs
    ):
        self.data_dir             = data_dir
        self.set_name             = set_name
        self.classes              = classes
        self.image_names          = [l.strip().split(None, 1)[0] for l in open(os.path.join(data_dir, 'ImageSets', 'Main', set_name + '.txt')).readlines()]  # image_names 에는 ['1.img', '2.img', '3.img', .. ] 이런 것들이 들어 있음.
        self.image_extension      = image_extension 
        self.skip_truncated       = skip_truncated
        self.skip_difficult       = skip_difficult

        self.labels = {}
        for key, value in self.classes.items():
            self.labels[value] = key

        super(PascalVocGenerator, self).__init__(**kwargs)

    def size(self):
        return len(self.image_names)

    def num_classes(self):
        return len(self.classes)

    def name_to_label(self, name):
        return self.classes[name]

    def label_to_name(self, label):
        return self.labels[label]

    def image_aspect_ratio(self, image_index):
        path  = os.path.join(self.data_dir, 'JPEGImages', self.image_names[image_index] + self.image_extension)
        try:
            image = Image.open(path)
        except Exception as ex:
            print(path)
        return float(image.width) / float(image.height)

    def load_image(self, image_index): # 이미지 번호 (전체 데이터 중 i 번째) 를 받아서, 해당 이미지의 경로를 생성, 해당 이미지를 로드
        path = os.path.join(self.data_dir, 'JPEGImages', self.image_names[image_index] + self.image_extension)
        return read_image_bgr(path)

    def __parse_annotation(self, element):
        truncated = _findNode(element, 'truncated', parse=int)
        difficult = _findNode(element, 'difficult', parse=int)

        class_name = _findNode(element, 'name').text
        if class_name not in self.classes:
            raise ValueError('class name \'{}\' not found in classes: {}'.format(class_name, list(self.classes.keys())))

        box = np.zeros((1, 5))
        box[0, 4] = self.name_to_label(class_name)

        bndbox    = _findNode(element, 'bndbox')
        box[0, 0] = _findNode(bndbox, 'xmin', 'bndbox.xmin', parse=float) + 1
        box[0, 1] = _findNode(bndbox, 'ymin', 'bndbox.ymin', parse=float) + 1
        box[0, 2] = _findNode(bndbox, 'xmax', 'bndbox.xmax', parse=float) - 1
        box[0, 3] = _findNode(bndbox, 'ymax', 'bndbox.ymax', parse=float) - 1

        return truncated, difficult, box

    def __parse_annotations(self, xml_root):
        size_node = _findNode(xml_root, 'size')
        width     = _findNode(size_node, 'width',  'size.width',  parse=float)
        height    = _findNode(size_node, 'height', 'size.height', parse=float)

        boxes = np.zeros((0, 5))
        for i, element in enumerate(xml_root.iter('object')):
            try:
                truncated, difficult, box = self.__parse_annotation(element)
            except ValueError as e:
                raise_from(ValueError('could not parse object #{}: {}'.format(i, e)), None)

            if truncated and self.skip_truncated:
                continue
            if difficult and self.skip_difficult:
                continue
            boxes = np.append(boxes, box, axis=0)
        return boxes

    def load_annotations(self, image_index):
        filename = self.image_names[image_index] + '.xml'
        try:
            tree = ET.parse(os.path.join(self.data_dir, 'Annotations', filename))
            return self.__parse_annotations(tree.getroot())
        except ET.ParseError as e:
            raise_from(ValueError('invalid annotations file: {}: {}'.format(filename, e)), None)
        except ValueError as e:
            raise_from(ValueError('invalid annotations file: {}: {}'.format(filename, e)), None)

- load_image(index) : 
  - read_image_bgr(path) : 이미지 1개를 적절히 읽어와 return
  

### Training

In [ ]:
def create_yolo(inputs, training=True, num_classes=20, weights=None, *args, **kwargs):
    if training:
        image, gt_boxes = inputs
    else:
        image = inputs

    coord_loss, object_loss, noobject_loss, cls_loss, output = Yolo(training=training, num_classes=num_classes, weights=weights)([image, gt_boxes])

    model = tf.keras.Model(inputs=inputs, outputs=[coord_loss, object_loss, noobject_loss, cls_loss,output], name="yolov1")
    return model

In [ ]:
import argparse
import os

import keras
import keras.preprocessing.image

from core.models.model import create_yolo
from core.preprocessing import PascalVocGenerator

# os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
# os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
# os.environ["CUDA_VISIBLE_DEVICES"] = '0'

def create_model():
    image = keras.layers.Input((448, 448, 3))
    # image = keras.layers.Input((None, None, 3))
    gt_boxes = keras.layers.Input((7, 7, 25))
    return create_yolo([image, gt_boxes], num_classes=20, weights=None)

'''
def parse_args():
    """Parse input arguments."""
    parser = argparse.ArgumentParser(description='Tensorflow Faster R-CNN demo')

    subparsers = parser.add_subparsers(help='Arguments for specific dataset types.', dest='dataset_type')
    subparsers.required = True

    pascal_parser = subparsers.add_parser('pascal')
    pascal_parser.add_argument('pascal_path', help='Path to dataset directory (ie. /tmp/VOCdevkit).', default='/home/syh/train_data/VOCdevkit/VOC2007')
    parser.add_argument('--root_path', help='Size of the batches.', default= os.path.join(os.path.expanduser('~'), 'keras_yolo'), type=str)

    parser.add_argument('--batch-size', help='Size of the batches.', default=4, type=int)

    args = parser.parse_args()

    return args
'''


DATA_PATH = '/home/syh/train_data/VOCdevkit/VOC2007'
BATCH_SIZE = 16

if __name__ == '__main__':
    # parse arguments
    args = parse_args()

    # create the model
    print('Creating model, this may take a second...')
    model = create_model()

    # compile model (note: set loss to None since loss is added inside layer)
    model.compile(loss=None, optimizer=keras.optimizers.adam(lr=1e-5, clipnorm=0.001))

    # print model summary
    model.summary()

    # create image data generator objects
    train_image_data_generator = keras.preprocessing.image.ImageDataGenerator(
        rescale=1.0 / 255.0,
        horizontal_flip=True,
        vertical_flip=True,
        width_shift_range=0.1,
        height_shift_range=0.1,
        zoom_range=0.1,
    )
    test_image_data_generator = keras.preprocessing.image.ImageDataGenerator(
        rescale=1.0 / 255.0,
    )

    # create a generator for training data
    train_generator = PascalVocGenerator(
        DATA_PATH,
        'train', # 데이터들의 목록이 저장되어 있는 DATA_PATH/train.txt 읽어오기
        batch_size=BATCH_SIZE,
        transform_generator = train_image_data_generator
    )

    # create a generator for testing data
    test_generator = PascalVocGenerator(
        DATA_PATH,
        'test', # 데이터들의 목록이 저장되어 있는 DATA_PATH/test.txt 읽어오기
        batch_size=BATCH_SIZE,
        transform_generator = test_image_data_generator
    )

    # start training
    model.fit_generator(
        generator=train_generator,
        steps_per_epoch=len(train_generator.image_names) // args.batch_size,
        epochs=100,
        verbose=1,
        validation_data=test_generator,
        validation_steps=len(test_generator.image_names) // args.batch_size,
        callbacks=[
            keras.callbacks.ModelCheckpoint(os.path.join(args.root_path, 'snapshots/yolo(v1)_voc_best.h5'), monitor='val_loss', verbose=1, mode='min', save_best_only=True),
            keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10, verbose=1, mode='auto', min_delta=0.0001, cooldown=0, min_lr=0),
        ],
    )

    # store final result too
    model.save('snapshots/yolo(v1)_voc_best.h5')

    '''
    cd tools
    python train_yolo.py pascal /home/syh/train_data/VOCdevkit/VOC2007
    '''

### Testing